In [21]:
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#CountVectorizer --> Bag of words
#TfidfVectorizer --> Tf-IDF
#gensim.models.Word2Vec --> word embeddings (word2vec)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from collections import Counter

In [20]:
df = pd.read_csv('preprocessed_data.csv')
df.fillna('', inplace=True)
X = df['cleaned_reviews']
Y = df['Label']

In [26]:
# bag of words
bow = CountVectorizer()
X_bow = bow.fit_transform(X)
# TF-IDF
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X)
#load the model 
w2v = Word2Vec.load('./trained-word2vec-studentReviews.model')
#creating sentence vector:
def sentence_to_vector(sentence, model):
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if not word_vectors:  # If the sentence contains no words in the model's vocabulary
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Create vectors for all sentences
X_vectors = np.array([sentence_to_vector(sentence, w2v) for sentence in X])

In [28]:
X_vectors.shape

(107018, 100)

In [ ]:
# # Prepare vocabulary (top 1200 most frequent words)
# all_words = [token for seq in X for token in seq.split()]  # Get all words from cleaned reviews
# word_counts = Counter(all_words)
# most_frequent_words = [word for word, count in word_counts.most_common(1200)]  # Select top 1200 words
# class Sequencer():
    
#     def __init__(self,
#                  all_words,
#                  max_words,
#                  seq_len,
#                  embedding_matrix
#                 ):
        
#         self.seq_len = seq_len
#         self.embed_matrix = embedding_matrix
#         """
#         temp_vocab = Vocab which has all the unique words
#         self.vocab = Our last vocab which has only most used N words.
    
#         """
#         temp_vocab = list(set(all_words))
#         self.vocab = []
#         self.word_cnts = {}
#         """
#         Now we'll create a hash map (dict) which includes words and their occurencies
#         """
#         for word in temp_vocab:
#             # 0 does not have a meaning, you can add the word to the list
#             # or something different.
#             count = len([0 for w in all_words if w == word])
#             self.word_cnts[word] = count
#             counts = list(self.word_cnts.values())
#             indexes = list(range(len(counts)))
        
#         # Now we'll sort counts and while sorting them also will sort indexes.
#         # We'll use those indexes to find most used N word.
#         cnt = 0
#         while cnt + 1 != len(counts):
#             cnt = 0
#             for i in range(len(counts)-1):
#                 if counts[i] < counts[i+1]:
#                     counts[i+1],counts[i] = counts[i],counts[i+1]
#                     indexes[i],indexes[i+1] = indexes[i+1],indexes[i]
#                 else:
#                     cnt += 1
        
#         for ind in indexes[:max_words]:
#             self.vocab.append(temp_vocab[ind])
                    
#     def textToVector(self,text):
#         # First we need to split the text into its tokens and learn the length
#         # If length is shorter than the max len we'll add some spaces (100D vectors which has only zero values)
#         # If it's longer than the max len we'll trim from the end.
#         tokens = text.split()
#         len_v = len(tokens)-1 if len(tokens) < self.seq_len else self.seq_len-1
#         vec = []
#         for tok in tokens[:len_v]:
#             try:
#                 vec.append(self.embed_matrix[tok])
#             except Exception as E:
#                 pass
        
#         last_pieces = self.seq_len - len(vec)
#         for i in range(last_pieces):
#             vec.append(np.zeros(100,))
        
#         return np.asarray(vec).flatten()

# # Create Sequencer object
# sequencer = Sequencer(
#     all_words=all_words,
#     max_words=1200,
#     seq_len=15,
#     embedding_matrix=w2v.wv  # Assuming you have a pre-trained word embedding matrix (model.wv)
# )

# # Convert text reviews to sequence vectors
# X_vectors = []
# for review in X:
#     # Convert each review to a sequence vector using sequencer.textToVector
#     vector = sequencer.textToVector(review)
#     X_vectors.append(vector)

In [30]:
pca_file = 'D:/Study/College/IV Sem/Mini Project/Codes/Trained Models/pca-model.sav'
pca_model = pickle.load(open(pca_file, 'rb'))
X_vectors_pca = pca_model.transform(X_vectors)

In [34]:
#test_size = 0.2 --> 20% of the data goes for testing and the remaing 80% goes for traning
#random_state = 42 --> ensures that the data is split in the same random way across all code runs-> this ensures efficient debugging
X_train_bow, X_test_bow, Y_train, Y_test = train_test_split(X_bow, Y, test_size=0.2, random_state=42)
X_train_tfidf, X_test_tfidf,_, _ = train_test_split(X_tfidf, Y, test_size=0.2, random_state=42)
X_train_w2v, X_test_w2v,_, _ = train_test_split(X_vectors_pca, Y, test_size=0.6, random_state=42)

In [35]:
#SVM
#SVM performs better when the data is standarised or normalised, but in word2vec -> data is scaled
scaler = StandardScaler()

X_train_w2v_scaled = scaler.fit_transform(X_train_w2v)
X_test_w2v_scaled = scaler.fit_transform(X_test_w2v)

In [36]:
svm_w2v = SVC(kernel='poly', max_iter=10000)
svm_w2v.fit(X_train_w2v_scaled, Y_train)

ValueError: Found input variables with inconsistent numbers of samples: [42807, 85614]

In [16]:
svm_pred_w2v = svm_w2v.predict(X_test_w2v)

In [17]:
print('SVM(Word2Vec) accurancy: ', accuracy_score(Y_test, svm_pred_w2v))

SVM(Word2Vec) accurancy:  0.7412633152681741
